# Crafting Adversarial samples with text for LSTM

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import tensorflow as tf
from tqdm import tqdm
import re
import string

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM, Activation, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.text import Tokenizer

import tensorflow_datasets as tfds
tfds.disable_progress_bar()



[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Dataset

We will be using IMDB review data set that can be classified as either a positive-negative review.

The data is available through Keras for retrieval. We can limit the total number of words in vocabulary.

In [6]:
tensor_train, tensor_test = tfds.load('imdb_reviews', split=['train','test'])
imdb_train = pd.DataFrame(list(tfds.as_numpy(tensor_train)))
imdb_test = pd.DataFrame(list(tfds.as_numpy(tensor_test)))
print("Shape of Train split: ", imdb_train.shape)
print("Shape of Test split: ", imdb_test.shape)
# with pd.set_option('display.max_colwidth', 100):
imdb_train.head()

Shape of Train split:  (25000, 2)
Shape of Test split:  (25000, 2)


,label,text
0,1,"b""As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels..."
1,1,"b""Oh yeah! Jenna Jameson did it again! Yeah Baby! This movie rocks. It was one of the 1st movies..."
2,1,"b""I saw this film on True Movies (which automatically made me sceptical) but actually - it was g..."
3,1,"b'This was a wonderfully clever and entertaining movie that I shall never tire of watching many,..."
4,1,"b'I have no idea what the other reviewer is talking about- this was a wonderful movie, and creat..."


In [ ]:
imdb_train.label.value_counts()

In [ ]:
imdb_test.label.value_counts()

In [ ]:
%%time
new_text = imdb_train.text.apply(lambda x : x.decode('utf-8')).apply(word_tokenize).explode()
new_text.unique().shape[0]

In [ ]:
# ### Decreasing the size of test set.
# np.random.seed(10)
# idx = np.random.choice(imdb_test.shape[0],2000)
# imdb_test = imdb_test.iloc[idx]
# imdb_test.shape

## Data pre-processing

In [ ]:
eng_stopwords = stopwords.words('english')
stopwords_dict = Counter(eng_stopwords)

def remove_stopwords(text):
    text = ' '.join([word for word in word_tokenize(text.lower()) if word not in stopwords_dict])
    return text

def html_tags(text):
    text = re.sub('\<.*?\>+', ' ', text)
    return text

def remove_punctuation(text):
#     punctuations = string.punctuation.replace('\'','')+'\t\n'
    punctuations = string.punctuation+'\t\n'
    text = re.sub('[%s]' % re.escape(punctuations), ' ', text)
    return text

In [ ]:
%%time
imdb_train['ptext'] = imdb_train.text.apply(lambda x : x.decode('utf-8'))
imdb_train['ptext'] = imdb_train.ptext.apply(html_tags)
imdb_train['ptext'] = imdb_train.ptext.apply(remove_punctuation)
imdb_train['ptext'] = imdb_train.ptext.apply(remove_stopwords)
imdb_test['ptext'] = imdb_test.text.apply(lambda x : x.decode('utf-8'))
imdb_test['ptext'] = imdb_test.ptext.apply(html_tags)
imdb_test['ptext'] = imdb_test.ptext.apply(remove_punctuation)
imdb_test['ptext'] = imdb_test.ptext.apply(remove_stopwords)
imdb_train.head()

In [ ]:
%%time
new_text = imdb_train.ptext.apply(word_tokenize).explode()
new_text.unique().shape[0]

In [ ]:
%%time

# Cut texts after this number of words (among top max_features most common words)
max_features = 80000 

# Define tokenizer 
tokenizer = Tokenizer(num_words=max_features,
                      lower=True)
#                       oov_token="<unk>")

# Fit the 
tokenizer.fit_on_texts(imdb_train.ptext)

# Use the '0' index for the padding character
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

In [ ]:
x_train = tokenizer.texts_to_sequences(imdb_train.ptext)
y_train = imdb_train.label.values

x_test = tokenizer.texts_to_sequences(imdb_test.ptext)
y_test = imdb_test.label.values

In [ ]:
print("Train data review statistics:")
pdlen = pd.Series(np.array([len(x) for x in x_train]))
print(pdlen.describe())
print()
print("Test data review statistics:")
pdlen = pd.Series(np.array([len(x) for x in x_test]))
print(pdlen.describe())
# print("Average number of words in each review:", lens.max())

We need to one-hot encode the labels, to use probabilities/logits for different classes

In [ ]:
print("One-hot encoding of labels")
y_train_oe = to_categorical(y_train, 2)
y_test_oe = to_categorical(y_test, 2)
print('train labels shape:',y_train.shape)
print('test labels shape:',y_test.shape)

Keras Embedding layer expects the input to have similar length for each review.
So we either need to pad or truncate the reviews as necessary.

We are padding/truncating at the end of the review.

In [ ]:
maxlen = 120

x_train = sequence.pad_sequences(x_train, padding='post', truncating='post', maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, padding='post', truncating='post', maxlen=maxlen)

print('train data shape:', x_train.shape)
print('test data shape:', x_test.shape)

In [ ]:
tokenizer.sequences_to_texts(x_train[0:2])

In [ ]:
print("Setting up model-specific variables...")
K.clear_session()
batch_size = 64
embedding_size = 256
lstm_size = 128
val_split = 0.2
epochs = 15
num_classes = 2

## Model Training & Evaluation

In [75]:
seq_encode = Input(shape=(maxlen,))
embeddings = Embedding(max_features, embedding_size)(seq_encode)
lstm_out = LSTM(lstm_size)(embeddings)
dense_out = Dense(num_classes)(lstm_out)
out = Activation('softmax')(dense_out)

In [76]:
optimizer = Adam(lr=1e-3, decay=1e-3)
imdb_clf = Model(inputs=seq_encode, outputs=out)
imdb_clf.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
imdb_clf.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 120)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 120, 256)          20480000  
_________________________________________________________________
lstm (LSTM)                  (None, 128)               197120    
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
_________________________________________________________________
activation (Activation)      (None, 2)                 0         
Total params: 20,677,378
Trainable params: 20,677,378
Non-trainable params: 0
_________________________________________________________________


In [77]:
train_history = imdb_clf.fit(x_train, y_train_oe,
                             validation_data=(x_test, y_test_oe),
#                              validation_split=val_split,
                             batch_size=batch_size,
                             epochs=epochs
                             )

Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 150s 6ms/sample - loss: 0.6352 - accuracy: 0.6332 - val_loss: 0.5728 - val_accuracy: 0.7394
Epoch 2/15
25000/25000 [==============================] - 146s 6ms/sample - loss: 0.6118 - accuracy: 0.6807 - val_loss: 0.6790 - val_accuracy: 0.6436
Epoch 3/15
25000/25000 [==============================] - 146s 6ms/sample - loss: 0.5232 - accuracy: 0.7526 - val_loss: 0.5556 - val_accuracy: 0.7181
Epoch 4/15
25000/25000 [==============================] - 146s 6ms/sample - loss: 0.5241 - accuracy: 0.7512 - val_loss: 0.5569 - val_accuracy: 0.7315
Epoch 5/15
25000/25000 [==============================] - 147s 6ms/sample - loss: 0.3464 - accuracy: 0.8632 - val_loss: 0.5653 - val_accuracy: 0.7080
Epoch 6/15
25000/25000 [==============================] - 147s 6ms/sample - loss: 0.2322 - accuracy: 0.9160 - val_loss: 0.4182 - val_accuracy: 0.8290
Epoch 7/15
25000/25000 [==========================

In [78]:
loss, accuracy = imdb_clf.evaluate(x_test, y_test_oe)
print('Accuracy over Test data:', accuracy)

25000/25000 [==============================] - 35s 1ms/sample - loss: 0.7297 - accuracy: 0.8269
Accuracy over Test data: 0.82692


In [79]:
loss, accuracy = imdb_clf.evaluate(x_train, y_train_oe)
print('Accuracy over Train data:', accuracy)

25000/25000 [==============================] - 35s 1ms/sample - loss: 0.0226 - accuracy: 0.9957
Accuracy over Train data: 0.99572


In [113]:
_, accuracy = imdb_clf.evaluate(x_test, y_test_oe)
print('Accuracy over Test data:', accuracy)

_, accuracy = imdb_clf.evaluate(x_train, y_train_oe)
print('Accuracy over Train data:', accuracy)

25000/25000 [==============================] - 33s 1ms/sample - loss: 0.7297 - accuracy: 0.8269
Accuracy over Test data: 0.82692
25000/25000 [==============================] - 33s 1ms/sample - loss: 0.0226 - accuracy: 0.9957
Accuracy over Train data: 0.99572


## Saving the model

In [81]:
imdb_clf.save("saved_models/imdb_compiled_clf_120dim.h5")

## Load the model

In [24]:
# imdb_clf = load_model("saved_models/imdb_compiled_clf_70dim.h5")

## Retrieve Embeddings for all the words in the Vocabulary

In [82]:
vocab_embeddings = imdb_clf.layers[1].embeddings.numpy()
print("Shape of the generated embeddings: ",vocab_embeddings.shape)

Shape of the generated embeddings:  (80000, 256)


## Keras function to extract embeddings for samples

In [83]:
get_embeddings = K.function([imdb_clf.layers[0].input],
                                  imdb_clf.layers[1].output)

print("Testing the embedding function with a single sample...")
test_embed = get_embeddings(x_test[0])
print("Shape of generated embeddings:",test_embed.shape)

Testing the embedding function with a single sample...
Shape of generated embeddings: (120, 256)


## Defining Submodel - from Embeddings to logits

In [84]:
### Defining necessary layers
embed_input = Input(shape=test_embed.shape)
embed_lstm = LSTM(lstm_size, dropout=0.2, recurrent_dropout=0.2)(embed_input)
embed_dense = Dense(num_classes)(embed_lstm)

### Define model with Embedding inputs and Logit outputs
embed_model = Model(inputs=embed_input, outputs=embed_dense)

### Transferring the trained weights from our IMDB Classifier model (imdb_clf)
embed_model.layers[1].set_weights(imdb_clf.layers[2].get_weights())
embed_model.layers[2].set_weights(imdb_clf.layers[3].get_weights())
# embed_model.summary()

## Adversarial crafting

### Calculate Jacobian matrix for all the words in the input

In [85]:
def compute_input_jacobian(x, y, model):
    x_embed = get_embeddings(x)
    x_tensor = tf.convert_to_tensor(x_embed.reshape(-1,maxlen,embedding_size), tf.float32)
    x_var = tf.Variable(x_tensor, dtype=tf.float32)

    with tf.GradientTape(watch_accessed_variables=False) as tape:
        tape.watch(x_var)
        # Get logits
        pred_y = model(x_var)

    # Calculate gradients
    x_gradients = tape.batch_jacobian(pred_y, x_var).numpy()
    print("Shape of the Jacobian:", x_gradients.shape)

    return x_gradients

In [86]:
def craft_sample(x, y, x_gradient, max_changes=maxlen):

    x_copy = x.copy()
    pred = np.argmax(imdb_clf.predict_on_batch(x_copy.reshape(-1,maxlen)))
    if pred != y :
        return x, 0

    for word in range(max_changes):

        word_grad = x_gradient[y, word]

        jac_sign = np.sign(word_grad)
        vocab_sign = np.sign(word_grad - vocab_embeddings)
        
        sum_up = np.absolute(np.add.reduce(vocab_sign - jac_sign, axis=1))
        sum_up[0] = 1000
        match_word = np.argmin(sum_up)
        x_copy[word] = match_word

        pred = np.argmax(imdb_clf.predict_on_batch(x_copy.reshape(-1,maxlen)))
        if pred != y : break

    return  x_copy, word

In [131]:
np.random.seed(10)

num_samples_class = 100

crafted_x = []
num_changes = []

idx0 = np.random.choice(np.argwhere(y_train == 0).reshape(-1,), num_samples_class, replace=False)
idx1 = np.random.choice(np.argwhere(y_train == 1).reshape(-1,), num_samples_class, replace=False)
idx = np.concatenate((idx0,idx1))
# np.random.shuffle(idx)

xs, ys, ys_oe = x_train[idx].copy(), y_train[idx].copy(), y_train_oe[idx].copy()

In [134]:
%%time

print("Calculating gradients...")
x_gradients = compute_input_jacobian(xs,ys,embed_model)
loss, acc = imdb_clf.evaluate(xs, ys_oe, verbose=0)
results = [('Original', 0, loss, acc)]
print("Loss and accuracy of selected samples: %.4f and %.4f"%(loss, acc))

Calculating gradients...
Shape of the Jacobian: (200, 2, 120, 256)
Loss and accuracy of selected samples: 0.0068 and 1.0000
CPU times: user 1min 8s, sys: 0 ns, total: 1min 8s
Wall time: 1min 6s


In [135]:
print("Crafting adversarial samples...")

Crafting adversarial samples...


In [136]:
print("Crafting with defined number of changes for each sample:")
max_changes = [10,20,30,40,60,80,100,120]

Crafting with defined number of changes for each sample:


In [137]:
loss_arr = []
acc_arr = []
perturbed_x = []
changes_track = []
for change in max_changes:
    print("Limiting the number of word changes to ",change," in each sample:")
    crafted_x = []
    num_changes = []
    
    for x, y, grad in tqdm(zip(xs, ys, x_gradients), total=xs.shape[0]):
        new_x , changes = craft_sample(x, y, grad, max_changes=change)
        crafted_x.append(new_x)
        num_changes.append(changes)
    
    crafted_x = np.array(crafted_x)
    num_changes = np.array(num_changes)
    
    perturbed_x.append(crafted_x)
    changes_track.append(num_changes)
    
    avg_changes = num_changes.mean()
    
    print("Evaluation for the crafted samples:")
    loss, acc = imdb_clf.evaluate(crafted_x, ys_oe)
    print("Average number of changes per sample:", avg_changes)
    results.append((str(change), avg_changes, loss, acc))
    loss_arr.append(loss)
    acc_arr.append(acc)
    

  0%|          | 0/200 [00:00<?, ?it/s]

Limiting the number of word changes to  10  in each sample:


100%|██████████| 200/200 [07:36<00:00,  2.28s/it]

Evaluation for the crafted samples:
 96/200 [=============>................] - ETA: 0s - loss: 0.1849 - accuracy: 0.8958

200/200 [==============================] - 0s 1ms/sample - loss: 0.1993 - accuracy: 0.8800


...

  0%|          | 0/200 [00:00<?, ?it/s]

Average number of changes per sample: 41.445
Limiting the number of word changes to  120  in each sample:


100%|██████████| 200/200 [29:34<00:00,  8.87s/it]

Evaluation for the crafted samples:
 96/200 [=============>................] - ETA: 0s - loss: 1.5526 - accuracy: 0.0000e+00

200/200 [==============================] - 0s 1ms/sample - loss: 1.3276 - accuracy: 0.0650
Average number of changes per sample: 42.85


In [138]:
# print("Crafting with variable changes for each sample:")

In [ ]:
# crafted_x = []
# num_changes = []
# for x, y, grad in tqdm(zip(xs, ys, x_gradients), total=xs.shape[0]):
#     new_x , changes = craft_sample(x, y, grad)
#     crafted_x.append(new_x)
#     num_changes.append(changes)

# crafted_x = np.array(crafted_x)
# num_changes = np.array(num_changes)

# print("Average number of changes per sample:", num_changes.mean())

# imdb_clf.evaluate(crafted_x, ys_oe)

In [ ]:
perturbed_x = np.array(perturbed_x)
changes_track = np.array(changes_track)

In [142]:
import pickle
with open('crafted x', 'ab') as fo:     
    pickle.dump(perturbed_x, fo)    

with open('Changes tracker', 'ab') as fo:     
    pickle.dump(changes_track, fo)  

In [143]:
results_df = pd.DataFrame(results,columns=['Max changes', 'Average changes', 'Loss', 'Accuracy'])
results_df.head(10)

,Max changes,Average changes,Loss,Accuracy
0,Original,0.000,0.006780,1.000
1,10,8.465,0.199264,0.880
2,20,16.550,0.466769,0.705
3,30,22.970,0.695085,0.575
4,40,28.045,0.872956,0.450
5,60,35.040,1.132386,0.290
6,80,39.375,1.277620,0.140
7,100,41.445,1.313585,0.080
8,120,42.850,1.327620,0.065


In [144]:
results_df.to_csv("Text clf attack 120dim results.csv")

# Read back pickled data

In [29]:
imdb_clf = load_model("saved_models/imdb_compiled_clf_120dim.h5")

In [20]:
import pickle

max_changes = [10,20,30,40,60,80,100,120]
with open('crafted x', 'rb') as fo:
    perturbed_x = pickle.load(fo)
    
with open('Changes tracker', 'rb') as fo:
    changes_track = pickle.load(fo)

In [24]:
np.random.seed(10)

num_samples_class = 100

crafted_x = []
num_changes = []

idx0 = np.random.choice(np.argwhere(y_train == 0).reshape(-1,), num_samples_class, replace=False)
idx1 = np.random.choice(np.argwhere(y_train == 1).reshape(-1,), num_samples_class, replace=False)
idx = np.concatenate((idx0,idx1))
# np.random.shuffle(idx)

xs, ys, ys_oe = x_train[idx].copy(), y_train[idx].copy(), y_train_oe[idx].copy()

In [25]:
perturbed_x.shape

(8, 200, 120)

In [87]:
def compare(tid):
    print("Original label:",ys[tid])
    imdb_clf.evaluate(xs[tid].reshape(-1,maxlen), ys_oe[tid].reshape(-1,2))
    print("Original tweet:")
    print(tokenizer.sequences_to_texts(xs[tid].reshape(-1,maxlen)))
    print()
    for i in range(len(max_changes)):
        max_change = max_changes[i]
        pred = np.argmax(imdb_clf.predict(perturbed_x[i,tid].reshape(-1,maxlen)))
        if pred != ys[tid]:
            print("For max changes:", max_change)
            print("Number of changes on the sample:",changes_track[i,tid]+1)
            print("Original label: ",ys[tid]," | Predicted label: ", pred)
            print(tokenizer.sequences_to_texts(perturbed_x[i,tid].reshape(-1,maxlen)))
            break
    print()


In [116]:
compare(1)

Original label: 0
1/1 [==============================] - 0s 12ms/sample - loss: 9.9990e-04 - accuracy: 1.0000
Original tweet:
['three stooges rocket travel 1959 first feature length film star stooges pretty bad makes three stooges go around world daze 1963 look like masterpiece stooges janitors rocket place climb rocket goes venus meet stuff including talking unicorn call uni bring back earth uni speaks like average pleasant person oh hello lovely planet hope like hilarious gags many scenes go stooges arrive back space film far story goes one told film makers picture continues another 10 minutes party nothing much happens stooges leave party film almost high point film end stooges sing dapper little song journey larry curly joe hit moe face two pies brutal another writer mentioned fine musical score huh music even noticed two classic tunes take romance goes']

For max changes: 20
Number of changes on the sample: 18
Original label:  0  | Predicted label:  1
['like see actually films man

In [115]:
compare(2)

Original label: 0
1/1 [==============================] - 0s 11ms/sample - loss: 0.0065 - accuracy: 1.0000
Original tweet:
['disappearance couple take family vacation new mexico find deep trouble taking detour main highway visit town seemingly abandoned 1948 unknown reasons town weaver seems harmless first tourist appeal family stranded overnight begin good reason suspect others experienced predicament fatal outcomes henleys watch blair witch project esquire video diary left town last victim ironically demonstrates best performance anyone movie although hamlin dey performances much better supporting casts emotional affect seems flat throughout movie disappearance appeal movie much suspense good direction however plot takes unexpected implausible turns seemingly make sense worse yet really understanding exactly going movie makes bizarre ending less tolerable appeared movie makers focused making stream suspenseful scenes threw away elements good story making plot development gradual expla

In [68]:
compare(3)

Original label: 0
Original tweet:
['wow plot film place much plot many things happen practically made head spin result none seemed particularly believable movie starts kay francis housewife living small town experience local theater ambitions going broadway big time actor arrives town pursues hopes give career boost husband worried shenanigans actor cad hubby bursts hits actor actor dies result convicted first degree murder manslaughter murder 1 pregnant need funds kay goes new york broadway jobs found forced take job even burlesque unable adequately care young daughter gives another woman raise however eventually find job real broadway play everything looks rosy jealous diva starring play hates inexplicable reason forces thrown play despondent makes way england becomes real star years later returns new york get kid child']

For max changes: 40
Number of changes on the sample: 35
Original label:  0  | Predicted label:  1
['also like really life plot like characters film one good know l

In [117]:
compare(4)

Original label: 0
1/1 [==============================] - 0s 12ms/sample - loss: 0.0071 - accuracy: 1.0000
Original tweet:
['sad lucian pintilie stop making movies get worse every time niki flo 2003 depressing stab camera unfortunate many movies made yearly romania worst get sent abroad e g chicago international film festival movie without plot acting script waste time money score 0 02 10 <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>']

For max changes: 30
Number of changes on the sample: 23
Original label:  0  | Predicted label:  1
['good like like watch like best people film actuall

In [118]:
compare(5)

Original label: 0
1/1 [==============================] - 0s 11ms/sample - loss: 0.0018 - accuracy: 1.0000
Original tweet:
['foolish implausible tale redeemed opening scene hard boiled police detective delivers nearly audible lines confirming greatest fears dead perhaps film would saved director forgone dazzling star power martinez favor sadly anonymous actor filled screen brief moment name hack tor street could salvage dishwater film less likely villain committing murder dropping stones quarry unsuspecting diver moment brief promise immense perhaps treated screen time obscure thespian ever sequel ill advised film <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>']

For max changes: 60
Number of changes on the sample: 50
Original label

In [123]:
np.argwhere(np.argmax(changes_track,axis=0)==3)

array([[  3],
       [  8],
       [ 14],
       [ 20],
       [ 33],
       [ 46],
       [ 52],
       [ 57],
       [ 64],
       [ 72],
       [ 76],
       [ 81],
       [ 92],
       [101],
       [104],
       [117],
       [138],
       [146],
       [152],
       [160],
       [161],
       [169],
       [177],
       [181],
       [189]])

In [128]:
compare(146)

Original label: 1
1/1 [==============================] - 0s 11ms/sample - loss: 0.0032 - accuracy: 1.0000
Original tweet:
['silverlake life documentary plain straightforward actually like home movie want dramatic illuminations see something else means tearjerker mean positive ways shows two men love afflicted aids affecting quality every day lives almost difficult say whether quality film undressed look ways respond admirable film actually one admirable sincere documents ever seen two men incredible integrity lives reduced basic parts makes hollow wood productions aids seem hip heartless men made movie one best reasons create something scene tom sings sunshine mark tells goodbye real thing <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>']

For max changes: 40
Number of changes on the sample: 31
Original label:  1  

In [121]:
compare(195)

Original label: 1
1/1 [==============================] - 0s 13ms/sample - loss: 0.0059 - accuracy: 1.0000
Original tweet:
['first saw movie freshman high school film stuck years soundtrack cinematography even dialogue somewhat bad acting educational purpose message behind important sin child teenager still high school really bad thing either problem tons girls knew children guess never watched great movie clearly get message behind taking chances sexual relationship girl get pregnant first time myth necessarily lose dreams take backseat future child think first movie clear message behind say <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>']

For max changes: 10
Number of changes on the sample: 4
Original label:  1  | Predicted label

In [122]:
compare(107)

Original label: 1
1/1 [==============================] - 0s 13ms/sample - loss: 0.2285 - accuracy: 1.0000
Original tweet:
['movie lots great actors actresses addressed noble issues full emotion direction done well storyline progresses quickly guess better watch 3 hour movie easy movie watch enjoy <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>']

For max changes: 10
Number of changes on the sample: 1
Original label:  1  | Predicted label:  0
['story lots great actors actresses ad